In [11]:
from dotenv import load_dotenv
load_dotenv()


True

In [12]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=[r"C:\Users\Afzal\Desktop\mcp\langraph_mcp\poject00\LangGraph_MCP\mcp_server.py"],
)

async with sse_client(url="http://localhost:8000/sse") as streams:
    async with ClientSession(*streams) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [13]:
for m in agent_response["messages"]:
    m.pretty_print()

================================ Human Message =================================

what's (3 + 5) x 12?
================================== Ai Message ==================================
Tool Calls:
  add (call_ikCKgk1FLgh6bMHGjk80M1PF)
 Call ID: call_ikCKgk1FLgh6bMHGjk80M1PF
  Args:
    a: 3
    b: 5
  multiply (call_9BP0pnrcjQmc3hp5ADEfChjg)
 Call ID: call_9BP0pnrcjQmc3hp5ADEfChjg
  Args:
    a: 8
    b: 12
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

The result of \( (3 + 5) \times 12 \) is 96.
